# HMM 2
## Specification
- Name: HMM with implemented Baum-Welch Learning
- Natebook Name: problem26.ipynb
- options: none
- input: filename passed as first parameter to main
- output: a text file. ( using print ( .... file=someFileObject) is a handy way to do this after you have opened someFileObject as a text file). I find it name to name these files by creating a string by concatenating the string named infile with ".out" ... rosalind_ba10a.txt.out ( for example).


## Description
These are drawn from material presented in Ch. 10 of Compeau and Pevzner.

We begin an exploration of models that provide us with inference into the hidden meaning lying within a sequence. Four HMM Rosalind examples provide a framework that will extend throughout the course. Note that these examples operate with probabilities rather than log-probabilities or log-odds scores. The concepts of the models are consistent, however.



## Inspection Intro

- The BioinformaticsModel class encapsulates the functionality for parsing input data and calculating emission probabilities using forward probabilities. The class is designed to maintain state and provide a clean interface for these calculations.

In [2]:
# python3
import sys
from math import *
import numpy as np

'''
Implement Baum-Welch Learning.
Input: A sequence of emitted symbols x = x1 . . . xn in an alphabet A, generated by a k-state HMM with
 unknown transition and emission probabilities, initial Transition and Emission matrices and a number of iterations I.
Output: A matrix of transition probabilities Transition and a matrix of emission probabilities Emission that
 maximizes Pr(x,π) over all possible transition and emission matrices and over all hidden paths π.

Sample Input:
10
--------
xzyyzyzyxy
--------
x	y	z
--------
A	B
--------
	A	B
A	0.019	0.981 
B	0.668	0.332 
--------
x	y	z
A	0.175	0.003	0.821 
B	0.196	0.512	0.293
Sample Output:
	A	B
A	0.000	1.000	
B	0.786	0.214	
--------
	x	y	z
A	0.242	0.000	0.758	
B	0.172	0.828	0.000
'''

class BaumWelch:
    def __init__(self):
        x, transition, emission, alphabet, states, iterNo = self.readFromFile()
        transition, emission = self.BaumWelchLearning(x, transition, emission, alphabet, states, iterNo)
        self.saveTransitionAndEmission(alphabet, states, transition, emission)

    def readFromFile(self):
        f = open('rosalind_ba10k.txt', 'r')
        data = f.read().strip().split()
        iterNo = int(data[0])
        x = data[2]
        ind = [i for i in range(len(data)) if '--------' == data[i]]
        alphabet = data[ind[1]+1:ind[2]]
        states = data[ind[2]+1:ind[3]]
        transition = np.zeros((len(states), len(states)), dtype = float)
        emission = np.zeros((len(states), len(alphabet)), dtype = float)
        for i in range(len(states)):
            transition[i, :] = [float(d) for d in data[ind[3]+len(states)+2+i*(len(states)+1):ind[3]+len(states)+1+(i+1)*(len(states)+1)]]
            emission[i, :] = [float(d) for d in data[ind[4]+len(alphabet)+2+i*(len(alphabet)+1):ind[4]+len(alphabet)+1+(i+1)*(len(alphabet)+1)]]
        return x, transition, emission, alphabet, states, iterNo

    def softDecode(self, xList, transition, emission):
        n = len(xList)
        l = transition.shape[0]
        forward = [[0 for _ in range(l)] for __ in range(n)]
        backward = [[0 for _ in range(l)] for __ in range(n)]
        for k in range(l):
            forward[0][k] = emission[k, xList[0]]/l
        for i in range(1, n):
            for k in range(l):
                forward[i][k] = sum([forward[i-1][kpre]*transition[kpre, k]*emission[k, xList[i]] for kpre in range(l)])
        fsink = sum(forward[n-1])

        for k in range(l):
            backward[n-1][k] = 1
        for i in range(n-2, -1, -1):
            for k in range(l):
                backward[i][k] = sum([backward[i+1][kpre]*transition[k, kpre]*emission[kpre, xList[i+1]] for kpre in range(l)])
        
        Pr = np.zeros((l, n), dtype = float)
        for i in range(n):
            for k in range(l):
                Pr[k, i] = forward[i][k]*backward[i][k]/fsink
        
        Pr2 = np.zeros((l, l, n-1), dtype = float)
        for k1 in range(l):
            for k2 in range(l):
                for i in range(n-1):
                    Pr2[k1, k2, i] = forward[i][k1]*transition[k1, k2]*emission[k2, xList[i+1]]*\
                    backward[i+1][k2]/fsink

        return Pr, Pr2
    
    def estimateParameters(self, xList, Pr, Pr2, nAlphabet):
        n = len(xList)
        l = Pr2.shape[0]
        transition = np.zeros((l, l), dtype = float)
        emission = np.zeros((l, nAlphabet), dtype = float)
        for k1 in range(l):
            for k2 in range(l):
                transition[k1, k2] = sum(Pr2[k1, k2, :])
        for k in range(l):
            for i in range(n):
                emission[k, xList[i]] += Pr[k, i]

        for i in range(l):
            sum1 = sum(transition[i,:])
            if 0 == sum1:
                transition[i,:] += 1/l
            else:
                transition[i,:] /= sum1
            sum2 = sum(emission[i,:])
            if 0 == sum2:
                emission[i,:] += 1/nAlphabet
            else:
                emission[i,:] /= sum2
        return transition, emission

    def BaumWelchLearning(self, x, transition, emission, alphabet, states, iterNo):
        x2ind = {alphabet[i]:i for i in range(len(alphabet))}
        xList = [x2ind[x[i]] for i in range(len(x))]
        for _ in range(iterNo):
            Pr, Pr2 = self.softDecode(xList, transition, emission)
            transition, emission = self.estimateParameters(xList, Pr, Pr2, len(alphabet))
        return transition, emission

    def saveTransitionAndEmission(self, alphabet, states, fullTransition, emission):
        f = open('rosalind_ba10k_out.txt', 'w')
        print(' '.join([' '] + states))
        f.write('\t'+'\t'.join(states) + '\n')
        for i in range(fullTransition.shape[0]):
            print(' '.join([states[i]] + ['{:.3f}'.format(a) for a in fullTransition[i, :]]))
            f.write('\t'.join([states[i]] + ['{:.3f}'.format(a) for a in fullTransition[i, :]]) + '\n')
        print('--------')
        f.write('--------'+'\n')
        print(' '.join([' '] + alphabet))
        f.write('\t'+'\t'.join(alphabet)+'\n')
        for i in range(emission.shape[0]):
            print(' '.join([states[i]] + ['{:.3f}'.format(a) for a in emission[i, :]]))
            f.write('\t'.join([states[i]] + ['{:.3f}'.format(a) for a in emission[i, :]])+'\n')
        f.close()

if __name__ == '__main__':
    BaumWelch()

  A B
A 0.080 0.920
B 0.352 0.648
--------
  x y z
A 0.256 0.521 0.223
B 0.386 0.311 0.302


## Inspection Results

# Q: Why does the calculate_emission_probability method use a two-dimensional list for forward_probabilities? (Connor)
- A: The method uses a two-dimensional list to store the forward probabilities at each step for each state. This structure is essential for implementing the dynamic programming approach to calculate the total probability efficiently.

# Q: How does the BioinformaticsModel class manage the complexity of handling multiple matrices (transition and emission)? (Ivana)
- A: The class uses dictionaries to store both transition and emission matrices, facilitating easy access and manipulation of these complex data structures within the class methods.